## Final Data Matching

Where we match the air quality, traffic, and weather data into one dataframe.

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import glob, json

from src.utils import haversine_dist
from src.utils.get_data import get_air_locations_df, get_traffic_locations_df 

### Load Air Quality, Weather, and Traffic Data

In [3]:
### Air Quality Data
aq_df = pd.read_feather("../01-data/processed/air_quality_data.feather")

### Weather data
weather_df = pd.read_feather("../01-data/processed/weather_data.feather")
weather_df["time"] = weather_df.time.dt.tz_localize("UTC")

### Traffic data
traffic_df = pd.read_feather("../01-data/processed/traffic_data.feather")\
                .rename(columns={"fecha":"time"})\
                        .loc[:,["time","nombre","cod_cent","id","intensidad","carga","ocupacion"]]
traffic_df = traffic_df.loc[traffic_df.intensidad+traffic_df.ocupacion>=0,:]
# Ensure it is in UTC
traffic_df["time"] = traffic_df.time.dt.tz_localize("Europe/Madrid",ambiguous="NaT",nonexistent="shift_forward")\
                                    .dt.tz_convert("UTC")
traffic_df = traffic_df.dropna(subset=["time"])
### Traffic locations
traffic_locations_df = get_traffic_locations_df()

### Air locations
air_locations_df = get_air_locations_df()


In [7]:
matched_dfs = []
for estacion_aire,lat,long in air_locations_df[["estacion_aire","latitud","longitud"]].values:
    # Compute distance between air stations and traffic stations in km
    km_distances = haversine_dist(lat,long,traffic_locations_df.latitud,traffic_locations_df.longitud)
    # Filter those stations that are within 500m
    traffic_stations_nearby = traffic_locations_df.loc[km_distances<=0.75,["cod_cent"]]\
                                .assign(km_dist=km_distances[km_distances<=0.75])\
                                    .set_index("cod_cent")
    # Weight the traffic intensity by the distance to the air station
    traffic_nearby_df = traffic_df\
        .loc[traffic_df.cod_cent.isin(traffic_stations_nearby.index)]\
            .join(
                traffic_stations_nearby,
                on="cod_cent",
                how="left"
            )
    traffic_nearby_df["weighted_intensity"] = traffic_nearby_df.intensidad*np.exp(-np.logaddexp(0, (traffic_nearby_df.km_dist-0.38)*15))
    avg_weighted_intensity = traffic_nearby_df.groupby("time").weighted_intensity.mean().rename("traffic_intensity")
    avg_traffic_load = traffic_nearby_df.groupby('time').carga.mean().rename("traffic_load")
    # Join traffic data to air quality data
    air_station_df = aq_df\
        .loc[(aq_df.estacion==estacion_aire)&(aq_df.time.isin(avg_weighted_intensity.index))]\
            .join(
                pd.concat([avg_weighted_intensity,avg_traffic_load],axis=1),
                on="time",
                how="left"
            )
    # Join weather data to air quality and traffic data
    air_station_df = air_station_df.join(
        weather_df.loc[weather_df.time.isin(air_station_df.time)].set_index("time"),
        on="time",
        how="left"
    )
    matched_dfs.append(air_station_df)
madrid_air_quality_data = pd.concat(matched_dfs)

In [9]:
madrid_air_quality_data.reset_index(drop=True).to_feather("../01-data/processed/madrid_air_quality_data.feather")

### Meteorologically-Normalized Air Quality data

Obtained after running the notebook `meteorological_normalization.ipynb` that uses the R package `rmweather` to normalize the air quality data of each station as obtained above.

In [42]:
paths = glob.glob("../01-data/processed/normalized/*.csv")
aq_station_datasets_names = json.load(open("../references/air_quality_data/aq_station_datasets_names.json","r"))
station_names_dict = {dataset_name:estacion_name for estacion_name,dataset_name in aq_station_datasets_names.items()}
dfs = []
for csv in paths:
    df = pd.read_csv(csv)
    dataset_name = csv.split("-normalized")[0].split("aq_weather")[-1].strip("_")
    df["station"] = station_names_dict[f"aq_weather_{dataset_name}"]
    dfs.append(df)
aq_normalized_df = pd.concat(dfs).reset_index(drop=True)
aq_normalized_df["date"] = pd.to_datetime(aq_normalized_df["date"]).dt.tz_localize("UTC")
aq_normalized_df = aq_normalized_df[["date","station"]+aq_normalized_df.columns.difference(["date","station"]).tolist()]
aq_normalized_df

,date,station,no2_ug_m3,no_ug_m3,o3_ug_m3,pm10_ug_m3,pm25_ug_m3
0,2014-01-01 00:00:00+00:00,Villaverde Alto,39.286335,27.755977,48.544164,NaN,NaN
1,2014-01-01 01:00:00+00:00,Villaverde Alto,38.053602,23.500236,45.735993,NaN,NaN
2,2014-01-01 02:00:00+00:00,Villaverde Alto,38.223136,26.980486,47.252548,NaN,NaN
3,2014-01-01 03:00:00+00:00,Villaverde Alto,38.999782,23.878077,45.773249,NaN,NaN
4,2014-01-01 04:00:00+00:00,Villaverde Alto,36.946239,28.463852,42.228830,NaN,NaN
...,...,...,...,...,...,...,...
1335254,2020-12-31 19:00:00+00:00,Arturo Soria,35.761318,11.257909,49.910452,NaN,NaN
1335255,2020-12-31 20:00:00+00:00,Arturo Soria,34.259901,12.317439,47.207111,NaN,NaN
1335256,2020-12-31 21:00:00+00:00,Arturo Soria,34.702838,12.058701,49.281182,NaN,NaN
1335257,2020-12-31 22:00:00+00:00,Arturo Soria,34.224388,12.958695,47.567630,NaN,NaN


In [ ]:
aq_normalized_df = aq_normalized_df.join(
    weather_df.set_index("time"),
    how="left",
    on="date"
)
# aq_normalized_df.to_csv("../01-data/processed/aq-weather_normalized.csv")
aq_normalized_df